In [1]:
import numpy as np
import pandas as pd
import re

from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [2]:
CLINICAL_DATA_PATH_1 = "../data/broad_tcga/clinical/gdac.broadinstitute.org_STAD.Clinical_Pick_Tier1.Level_4.2016012800.0.0.tar.gz"

RPPA_PATH = "../data/broad_tcga/rppa/gdac.broadinstitute.org_STAD.RPPA_AnnotateWithGene.Level_3.2016012800.0.0/STAD.rppa.txt"
METH_PATH = "../data/broad_tcga/methylation/gdac.broadinstitute.org_STAD.Merge_methylation__humanmethylation450__jhu_usc_edu__Level_3__within_bioassay_data_set_function__data.Level_3.2016012800.0.0.tar.gz"
MIRNA_PATH_HISEQ = "../data/broad_tcga/miRSeq/gdac.broadinstitute.org_STAD.Merge_mirnaseq__illuminahiseq_mirnaseq__bcgsc_ca__Level_3__miR_gene_expression__data.Level_3.2016012800.0.0.tar.gz"

RNA_PATH_HISEQ_v1 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseq__illuminahiseq_rnaseq__bcgsc_ca__Level_3__gene_expression__data.Level_3.2016012800.0.0.tar.gz'
RNA_PATH_GA_v1 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseq__illuminaga_rnaseq__bcgsc_ca__Level_3__gene_expression__data.Level_3.2016012800.0.0.tar.gz'
RNA_PATH_HISEQ_v2 = ''
RNA_PATH_HISEQ_normalized_v2 = '../data/broad_tcga/mRNASeq/gdac.broadinstitute.org_STAD.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes_normalized__data.Level_3.2016012800.0.0.tar.gz'

In [3]:
def load_rppa(path=RPPA_PATH) -> pd.DataFrame:
    df = pd.read_csv(RPPA, sep='\t', index_col=0, header=0)
    return df

def load_mirna(path=MIRNA_PATH_HISEQ) -> pd.DataFrame:
    df = pd.read_csv(path, sep='\t', header=0, index_col=0)
    df = df.iloc[1:, (df.loc['miRNA_ID'] == 'reads_per_million_miRNA_mapped').values]
    return df

def load_mrna(path=RNA_PATH_HISEQ_normalized_v2) -> pd.DataFrame:
    df = pd.read_csv(path, compression='gzip',
                    index_col=0, header=0, sep='\t').iloc[1:, :]
    return df

def load_clinical(path=CLINICAL_DATA_PATH_1) -> pd.DataFrame:
    df = pd.read_csv(path, index_col=0, sep='\t')
    return df

def load_data(types: list = ['clinical', 'mRNA', 'miRNA']) -> dict:
    '''
    Loads dataframes into dict by data type.
    
    :param types
    :return dict
    '''
    
    raise NotImplementedError("Need to add at least copynumber, maybe mutations, test rppa.")
    # mutations seems big file, might take a while
    # methylation is with similar issue
    
    types_l = [s.lower() for s in types]  # lower strings' chars
    d = {}
    d['clinical'] = load_clinical()  # need to load clinical anyways
    
    if 'mrna' in types_l:
        d['mrna'] = load_mrna()
    if 'mirna' in types_l:
        d['mirna'] = load_mirna()
    
    
    return d

In [4]:
def get_ids_by_subtype(df: pd.DataFrame, subtype: str or list):
    '''
    '''
    
    histotypes = df.loc['histological_type'].iloc[0].to_list()
    
    short_names = {'src': r'signet ring type',
                  'diffuse': r'stomach, adenocarcinoma, diffuse type'}
    
    if type(subtype) == list:
        pass
    
    elif type(subtype) == str:
        assert subtype in short_names.keys()
        subtype = [subtype]
    
    else:
        raise ValueError("subtype string not recognized.")
    
    ids = []
    for subt in subtype:
        assert subt in short_names.keys()
        
        pattern = short_names[subt]
        subt_ids = df.columns[list(map(
            lambda x: bool(re.search(pattern, str(x))),
            histotypes))].to_list()
        ids += subt_ids
            
    return ids


def get_src_mask(df: pd.DataFrame, ids: list):
    '''
    Gets the boolean mask for samples corresponding to ids.
    
    :param df - dataframe in expected format (rows are omics' features, columns are tcga samples)
    :param ids - list of tcga ids of interest. E.g. ids of patients with SRC.
    '''
    rgx = r"|".join(ids)
    src_mask = list(map(lambda x: bool(re.search(rgx, x)),
                        map(lambda x: "-".join(x.lower().split("-")[:3]), df.columns)))
    return src_mask


# Perform Mann-Whitney U-test - nonparametric test for testing different expression for signet ring cells
# An equivalent to ANOVA test is Kruskal-Wallis one way analysis, which was employed by broad gdac,
# as a nonparametric, robust equivalent to one-way anova.
# Let's have a function that takes a dataframe and computes this test for histologic type, in 2 versions:
# kruskal-wallis and mann-whitney for src (with direction).

def associations_with_src(df: pd.DataFrame, src_ids: list, background_ids: list or None) -> dict:
    '''
    Computes dict of associations (test statistic and p-value) 
     of features in df with sample appartenance in src_ids.
    
    :param df
    :param src_ids
    '''
    
    src_mask = get_src_mask(df, src_ids)
    
    if background_ids is None:
        nonsrc_mask = [not i for i in src_mask]
    elif type(background_ids) == list:
        nonsrc_mask = get_src_mask(df, background_ids)
    else:
        raise TypeError("background_ids type not recognized.")
    
    res = {}
    for idx, row in df.iterrows():
        src_vals    = row[src_mask].astype('float64')
        nonsrc_vals = row[nonsrc_mask].astype('float64')
        
        try:
            mwu = mannwhitneyu(nonsrc_vals, src_vals, alternative='two-sided')
        except ValueError:
            # assume all nrs are 0 and equal
            res[idx] = [0, 1]
        
        mean_diff   = np.mean(src_vals) - np.mean(nonsrc_vals)
        median_diff = np.median(src_vals) - np.median(nonsrc_vals)
        
        res[idx] = [mean_diff, median_diff, mwu[0], mwu[1]]
    
    return res


def compute_src_statistics(df: pd.DataFrame, src_ids: list, background_ids: list or None) -> pd.DataFrame:
    '''
    Computes statistics for associating molecular features of SRC subtype.
    
    :param df - samples are tcga ids of all patients in background
    :param src_ids - list of ids of samples corresponding to patients with SRC tumors.
    '''
    
    r = associations_with_src(df, src_ids, background_ids)
    r = pd.DataFrame.from_dict(r, orient='columns').transpose()
    r.columns = ['mean_diff', 'median_diff', 'test-statistic', 'p-value']
    r.sort_values(['p-value'], ascending=True, inplace=True)
    
    # Add multiple test correction
    mtcorr = multipletests(r['p-value'],
                          alpha=0.05,
                          method='fdr_bh')
    r['p-value_corrected'] = mtcorr[1]
    r['p-value_reject']    = mtcorr[0]
    
    return r


def compute_allomics_src_statistics(target: str or list, 
                                   background: str or list or None = None,
                                   types: list = ['mrna', 'mirna']) -> pd.DataFrame:
    '''
    Computes statistics of association with SRC subtype for different multiomics data.
    
    :param paths - dict of omics_type -> Path()
    :param src_ids - list of ids of src type
    
    :return df with columns [feature, omics_type, test-statistic, p-value, p-value-corrected, p-value-reject]
    '''
    
    ### Load data
    dfs = load_data(types)
    
    ### Get ids for target and background groups for comparison
    src_ids = get_ids_by_subtype(dfs['clinical'], subtype=target)
    
    if background is None:
        background_ids = None
    else:
        background_ids = get_ids_by_subtype(dfs['clinical'], subtype=background)
    
    ### Compute results and collect
    res = {}
    for omics_type, df in dfs.items():
        
        if omics_type == 'clinical':
            continue
            
        df_res = compute_src_statistics(df, src_ids, background_ids)
        df_res['data_type'] = omics_type
        res[omics_type] = df_res
    
    ### Aggregate results in single dataframe
    df_final = pd.concat(res.values(), axis=0)
    df_final.sort_values(['p-value_corrected'], ascending=True, inplace=True)
    
    return df_final

In [5]:
# TODOS:
# - add methylation, RFPA, mutations, cnvs, whatever...
# - compare across various subtypes
# - add logistic regression to account for covariates (which ones ?...)  -  optional, showing off statistics.

target = ['src']
background = ['diffuse']
datatypes = ['mrna', 'mirna']  # to add: cnv, mutations, rfpa, ...

### Currently it ignores the covariates, might add with a multiple logistic regression.
res = compute_allomics_src_statistics(target, background, datatypes)

/home/dicer/miniconda3/envs/ortholog/lib/python3.7/site-packages/ipykernel_launcher.py:11: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,24

In [6]:
res

,mean_diff,median_diff,test-statistic,p-value,p-value_corrected,p-value_reject,data_type
LOC100271831|100271831,8.844388,0.000000,266.0,2.720926e-07,0.004066,True,mrna
CABP2|51475,0.148125,0.000000,300.0,3.960660e-07,0.004066,True,mrna
OR11H6|122748,0.171805,0.351900,204.0,2.317087e-06,0.015858,True,mrna
SNORA15|677803,0.291853,0.160600,244.0,4.941005e-06,0.025362,True,mrna
ZAR1L|646799,0.475750,0.372400,187.0,2.318463e-05,0.095205,False,mrna
SNORD2|619567,0.000000,0.000000,375.0,4.119258e-04,0.114293,False,mrna
SNORD29|9297,0.000000,0.000000,375.0,4.119258e-04,0.114293,False,mrna
SNORD28|9300,0.000000,0.000000,375.0,4.119258e-04,0.114293,False,mrna
SNORD27|9301,0.000000,0.000000,375.0,4.119258e-04,0.114293,False,mrna
SNORD26|9302,0.000000,0.000000,375.0,4.119258e-04,0.114293,False,mrna
